<a href="https://colab.research.google.com/github/jcandane/PhysicsI_Labs/blob/main/factor_NN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
import numpy as np
import scipy
import scipy.sparse.linalg
import scipy.sparse as sparse

try:
    import pyftt
except:
    !pip install pyftt
    import pyftt

try:
    import networkx as nx
except:
    !pip install networkx
    import networkx as nx

try:
    import network2tikz
    from network2tikz import plot
except:
    !pip install network2tikz ### https://pypi.org/project/network2tikz/
    import network2tikz
    from network2tikz import plot    

# premade def/functions codes

In [2]:
def su(N, JW=False):
    """
    j.candanedo 4/11/2023
    GIVEN:  N , dimension of Lie-Algebra
            JW, choice for Jordan-Wigner Transformation
    GET:    sparse-array (t=array-of-tuples, T=data-array, out_shape=dense-array-shape)
    """

    if JW:
        sparse_elements = int( N*(N-1) + N*(N+1)/2 - 1 )
        T = np.zeros(sparse_elements)
    else:
        sparse_elements = int( 2*N*(N-1) + N*(N+1)/2 - 1 )
        T = np.zeros(sparse_elements, dtype=complex)
    i = 0 ## sparse-array entry
    j = 0 ## generator-number
    t = np.zeros((sparse_elements,3), dtype=np.int32)
    for n in range(N-1): ## loop over Cartan-generators
        for m in range(n+1): ## loop over off-diagonals
            ## real generator
            t[i] = np.array([j, m, n+1])
            T[i] = 1.
            i+=1
            if JW:
                j+=1
            t[i] = np.array([j, n+1, m])
            T[i] = 1.
            i+=1
            j+=1

            ## imag generator
            if not JW:
                t[i] = np.array([j, m, n+1]) 
                T[i] = -1.j
                i+=1
                t[i] = np.array([j, n+1, m])
                T[i] =  1.j
                i+=1
                j+=1

        ## constant for Cartan-genorators.
        C = 1/np.sqrt( (n+1)*(n+2)/2 ) ## constant for Cartan-genorators.
        for m in range(n+1): ## loop over sparse Cartan-generator elements
            ## place-in Cartan-elements
            t[i] = np.array([j, m, m])
            T[i] = C
            i+=1
        ## last element of Cartan generator
        t[i] = np.array([j, n+1, n+1])
        T[i] = -(n+1)*C
        i+=1
        j+=1
    out_shape = np.array([j,N,N])
    return t, T, out_shape

def Dense_to_Sparse(A, tol=1.E-8):
    """
    Given:  A (arbitrary dimensional numpy.array)
            tol (tolerance, range of values which may be set to "0")
    Get:    sparse_array_object
    """
    indices=np.asarray(np.where(np.abs(A) >= tol))
    return pyftt.sparse_array(indices=indices, data=A[tuple( indices )], shape=np.asarray(A.shape), wellordered=True)

def integers_to_tuplesX(numbers, dense_shape):
    """
    GIVEN:  integer/integers (0d/1d int/int-numpy-array)
            dense_shape (1d numpy array, shape of the dense representation)
    GET:    list_of_tuples (2d int numpy int array)
    """

    ### 
    #dense_shape = np.roll(np.flip(dense_shape), 1)
    shapez    = np.roll(np.flip(dense_shape), 1)
    shapez[0] = 1
    shapez    = np.flip(np.cumprod(shapez))

    ### 
    numbers   = np.array([numbers]).flatten()
    out_tuple = np.zeros((len(numbers),len(dense_shape)), dtype=np.int64)
    for i in range(len(shapez)): ## for each column in new-shape
        out_tuple[:,i] = numbers // (shapez[i])
        numbers        = np.mod( numbers, (shapez[i]))
    return out_tuple

def tuples_to_integersX(list_of_tuples, dense_shape):
    """
    GIVEN:  list_of_tuples (1d/2d int numpy array)
            dense_shape (1d numpy array, shape of the dense representation)
    GET:    1d array of ints xor int (corresponding to pair-function)
    """
    dense_shape = np.roll(np.flip(dense_shape), 1)
    shapez    = 1*dense_shape
    shapez[0] = 1
    return list_of_tuples @ (np.flip(np.cumprod(shapez)))

def Reshape(list_of_tuples, old_shape, reshapinglist):

    ###### fit loose numbers into lists...[[1,0],[2,4],3] -> [[1,0],[2,4],[3]]
    for i in range(len(reshapinglist)):
        if isinstance(reshapinglist[i], int) or isinstance(reshapinglist[i], np.int32):
            reshapinglist[i] = np.array([reshapinglist[i]])
    ######

    ###### find:  [expansion] whether to expand (get tuples) xor contract (get integers).
    ######        [new_shape] shape of output sparse-tensor
    ######        [coln_shapes] shapes of each column (in new sparsetensor) to preform reshape 
    coln_shapes = []
    expansion   = []
    new_shape   = []
    for i in range(len(reshapinglist)):
        if len(reshapinglist[i]) > 1:
            if isinstance(reshapinglist[i][1], list):
                coln_shapes.append(reshapinglist[i][1])
                expansion.append(True)
                new_shape.append(reshapinglist[i][1])
            else:
                coln_shapes.append(old_shape[reshapinglist[i]])
                expansion.append(False)
                new_shape.append(np.prod(old_shape[reshapinglist[i]]))
        else:
            coln_shapes.append(old_shape[reshapinglist[i]])
            expansion.append(False)
            new_shape.append(np.prod(old_shape[reshapinglist[i]]))
    new_shape = np.asarray(new_shape)   
    ######

    ###### FINALLY, DETERMINE NEW_INDEX ARRAY
    new_index_array = np.zeros((len(new_shape), list_of_tuples.shape[1]) , dtype=np.int64)
    coln=0
    j=0
    for i in range(len(expansion)):
        if expansion[i]:
            l = (len(reshapinglist[i][1]))
            new_index_array[j:(j+l),:] = integers_to_tuplesX(list_of_tuples[reshapinglist[i][0],:], np.asarray(coln_shapes[i]) ).T
            j+=l
        else:
            new_index_array[j,:] = tuples_to_integersX(list_of_tuples[reshapinglist[i],:].T, coln_shapes[i])
            j+=1
    ######

    return new_index_array, new_shape

def cansum(D, label_D, S, label_S):
    """
    Contract a Dense and Sparse array (not affecting D nor S itself)
    GIVEN : D (Dense (regular) n-dimensional-numpy.array)
            label_D (idenification of each index in D)
            S (Sparse array, as defined in pyftt)
            label_S (idenification of each index in S)
    GET :   O (Dense (regular) n-dimensional-numpy.array)
            label_O (idenification of each index in O)
    """

    ### intersect labels
    Intersection = np.intersect1d(label_D, label_S)

    ### four labels...indices where...external and internal indices lie
    L_In_D = [ np.where(label_D==Intersection[i])[0][0] for i in range(len(Intersection)) ]
    L_Ex_D = np.delete( np.arange(len(label_D)) , L_In_D) 
    L_In_S = [ np.where(label_S==Intersection[i])[0][0] for i in range(len(Intersection)) ]
    L_Ex_S = np.delete( np.arange(len(label_S)) , L_In_S)

    ### get output label, shape, and tensor
    label_O = np.concatenate(( label_D[L_Ex_D], label_S[L_Ex_S] )) ## output label
    s_output = np.concatenate(( np.asarray(D.shape)[L_Ex_D], np.asarray(S.shape)[L_Ex_S] )) ## output-shape
    O        = np.zeros(s_output)   ## construct empty (zeros) O
    DS_divide= len(label_D[L_Ex_D]) ## divider for O into Dense-External & Sparse-External indices

    ### transpose & reshape (O, D, S)
    ExIndiv = len(L_Ex_D)
    DD= D.transpose( np.concatenate((L_Ex_D, L_In_D)) ).reshape((np.prod(D.shape[:ExIndiv]), np.prod(D.shape[ExIndiv:])))
    O = O.reshape(np.prod(O.shape[:DS_divide]), np.prod(O.shape[DS_divide:]))
    S_indices, S_shape = Reshape(S.indices, S.shape, [L_Ex_S, L_In_S])

    ### now sum over matrices
    for i in range(len(S.indices.T)):
        O[:,S_indices[0,i]] += DD[:,S_indices[1,i]]*S.data[i]
    #O[:,S.indices[0]] += D[:,S.indices[1]]*S.data ### does not work, why not??

    ### reshape O back to external-labels
    return O.reshape(s_output), label_O

def NN2_site(M, b):
    """
    GIVEN : M (size) [int]
            b (bitstring) [Boolean-np.array, int-np.array]
    GET   : O (superoperator index-array) [2D int-np.array]
    """
    b*=M
    min=1
    max=1
    O    = np.zeros((len(b)*(M)+1,len(b)), dtype=int)
    O[0] = b
    for i in range(len(b)):
        max+=M
        if b[i]==M: ## if R
            O[min:max,:] = b ## M xor 0
            O[min:(max),i] = np.arange(M)
        else: ## L
            O[min:max,:] = b ## M xor 0
            O[min:(max),i] = np.arange(1,M+1)
        min=max
    return O.T



In [45]:
def get_signature(G):
    """
    GIVEN : G (networkX graph object, with '.edges()' & '.degree' opt)
    GET   : signature (orientation for-all nodes in graph)
    **idea get unidirectional edge 2-tuples (between nodes)
    **associate each to 0 xor 1 data. transpose and NOT data
    **lexsort and partition via nodes into list
    """
    a = np.asarray(G.edges())
    R = np.random.randint(2, size=len(a)) ## random orientation on Graph

    aa = np.concatenate((a, a[:,[1,0]]), axis=0) ## swap-columns in a
    R  = np.concatenate((R, np.logical_not(R).astype(int)))
    R  = R[ np.lexsort( np.flip( aa.T , axis=0) , axis=0) ]

    ### make signature list
    j=0
    signature=[]
    for i in range(len(G.degree)):
        signature.append( R[j:(j+G.degree[i])] )
        j+=G.degree[i]
    return signature

def get_ncon(G):
    """
    GIVEN :  G (networkx graph object, s.t. G.edges() only include 1-set of edges
    GET   : ncon ( python-list of 1d np.arrays , tracing/contraction rule)
    """
    edges = np.asarray( G.edges() ) ## edges 2-tuple (sparse-adjacency-matrix)
    names = np.arange(1,len(edges)+1) ## edge enumeration ## each edge has an associtated alphabetical character
    edges = np.concatenate((edges, edges[:,[1,0]]), axis=0) ## swap-columns in a
    names = np.concatenate((names, names))

    ## reorder names in terms of edges
    names = names[ np.lexsort( np.flip( edges.T , axis=0) , axis=0) ]

    j,k=0,1
    ncon=[]
    for i in range(len(G.degree)):
        ncon.append( np.concatenate((-1*names[j:(j+G.degree[i])], [k,k+1])) )
        j+=G.degree[i]
        k+=2
    return ncon

##### iterative-Krylov-Solver


# Constructing the generic tensor-MPO (that factorizes $H_\text{NN2}$)

given
1.   $\texttt{N}\quad\quad\quad\quad$       (s.t. su(N))
2.   $\texttt{b}\quad\quad\quad\quad$     (bitstring for the tensor)
3.   $\texttt{H1}\quad\quad\quad\,\,$    (1-body Hamtilonian in sparse-matrix format, this is optional)

get
1.   sparse_array (pyftt object)





In [7]:
def get_tno_element(N, b, H1=None): ### do H1
    t   = su(N)     ## interactions enumerated....
    M   = N**2-1+1  ## super-operator length
    TNO = NN2_site(M, b) #tno_structure(M, ell)
    if H1==None:
        tno_= [] ## index-array
        tnod= [] ## data
    else:
        tno_= [H1.indices] ## cartesian-producted with bitstring!!!!!!!!!!!!!!!
        tnod= [H1.data]
    for I in range(1,TNO.shape[1]): ## start from 1 to negect nontrival 0 (xor 1-body Ham. here)

        A = TNO[:,I]
        ## find corners and save identity-matrix
        cornerdetect = np.logical_or( A==0, A==M ) ## corner-detector
        if np.all(cornerdetect):
            corner = np.zeros((len(A)+2,M), dtype=int)
            corner[:len(A),:]  = A[:,None]*np.ones(M,dtype=int)[None,:]
            corner[len(A),:]   = np.arange(M)
            corner[len(A)+1,:] = np.arange(M)
            tno_.append(corner.T)
            tnod.append(np.ones(M))
        
        ## save Lie-algebra (find non 0,M element: 1,2,3,4,5,...,M-2,M-1)
        else:
            q   = TNO[ np.logical_not(cornerdetect), I ][0]-1 ## search this
            L,R = np.searchsorted(t[0][:,0], q, side="left"), np.searchsorted(t[0][:,0], q, side="right")
            outt = ( t[0][:,1:][L:R,:] ) ## extract part of Lie-Algebra
            side = np.zeros((len(A)+2,len(outt)), dtype=int)
            side[:len(A),:]   = A[:,None]*np.ones(len(outt),dtype=int)[None,:]
            side[(len(A)):,:] = outt.T
            tno_.append(side.T)
            tnod.append(t[1][L:R])

    A = pyftt.sparse_array()
    A.indices = np.concatenate(tno_)
    A.data    = np.concatenate(tnod)
    A.shape   = np.concatenate(( M*np.ones(len(b),dtype=int), t[2][1:]))
    A.run()
    return A

get_tno_element(3, [0,1])

# Construct Tensor-Network from Graph/Network

given G ---> get TNO (with LoT and TrRx)
1.   $\texttt{N}\quad\quad\quad\quad$       (s.t. su(N))
2.   $\texttt{G}\quad\quad\quad\quad$       (NetworkX Graph Object)
3.   $\texttt{H1}\quad\quad\quad\,\,$    (1-body Hamtilonian in sparse-matrix format, this is optional)

In [47]:
def TNO(G, N, H1=None):
    signs = get_signature(G)
    ncon  = get_ncon(G) ## Tr-Rx

    ## build List-of-Tensors (LoT), a collection of sparse-tensors
    LoT = [ get_tno_element(N, signs[i], H1=H1) for i in range(len(signs)) ]

    return LoT, ncon

def greedy_path(G):
    return None

### save TNO as a: SciPy sparse.linalg.LinearOperator

greedy-algorthim (the MF of contractions)

In [83]:
𝙶𝚛𝚒𝚍𝟸𝚍 = nx.erdos_renyi_graph(8, 0.1) #𝚗𝚡.dodecahedral_graph() #𝚗𝚡.𝚐𝚛𝚒𝚍_𝚐𝚛𝚊𝚙𝚑((4,4))

nx.edge_bfs(𝙶𝚛𝚒𝚍𝟸𝚍, 2)
print( 𝙶𝚛𝚒𝚍𝟸𝚍.edges() )

#TNO(𝙶𝚛𝚒𝚍𝟸𝚍, 3)

[]


In [79]:
bfs = np.asarray( list( nx.bfs_edges(𝙶𝚛𝚒𝚍𝟸𝚍, 2) ) )

print(bfs)
print(bfs[:,1]) ## for each domain along bfs[:,0], count/sort the degree on bfs[:,1]

domains = np.concatenate( ([0], np.where( (bfs[:-1,0] - bfs[1:,0])!=0 )[0], [len(bfs)] ))
for d in range(len(domains)-1):
    print( [ 𝙶𝚛𝚒𝚍𝟸𝚍.degree[ bfs[i,1] ] for i in range(1+domains[d+1]-domains[d]) ] )
print(domains)

[[ 2  1]
 [ 2  3]
 [ 2  6]
 [ 1  0]
 [ 1  8]
 [ 3  4]
 [ 3 19]
 [ 6  5]
 [ 6  7]
 [ 0 10]
 [ 8  9]
 [ 4 17]
 [19 18]
 [ 5 15]
 [ 7 14]
 [10 11]
 [ 9 13]
 [17 16]
 [11 12]]
[ 1  3  6  0  8  4 19  5  7 10  9 17 18 15 14 11 13 16 12]
[3, 3, 3]
[3, 3, 3]
[3, 3, 3]
[3, 3, 3]
[3, 3]
[3, 3]
[3, 3]
[3, 3]
[3, 3]
[3, 3]
[3, 3]
[3, 3]
[3, 3]
[3, 3, 3]
[ 0  2  4  6  8  9 10 11 12 13 14 15 16 17 19]


# Applications

We would like to consider 4 graphs
1.   Path Graph $\quad\quad\quad\quad\,\left( \texttt{Path22 = nx.path_graph(22)} \right)$
2. Petersen Graph $\quad\quad\quad\!\left( \texttt{Petersen = nx.petersen_graph()} \right)$
3.   2D Grid-graph (4x4) $\quad\quad\!\!\left( \texttt{Grid2d = nx.grid_graph((4,4))} \right)$
4.   Paley Graph $\quad\quad\quad\quad\left( \texttt{Paley = nx.paley_graph(17)} \right)$



## Path Graph